In [ ]:
# -*- coding: utf-8 -*-
#  Copyright 2021 - 2024 United Kingdom Research and Innovation
#  Copyright 2021 - 2024 The University of Manchester
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Mariam Demir (UKRI-STFC)

### Create AcquisitionGeometry Using Dataset Metadata
This example shows how to use the dataset's metadata to manually create and visualise the AcquisitionGeometry, and the AcquisitionData. 

In [ ]:
import os
from cil.io import TIFFStackReader
from cil.framework import AcquisitionGeometry, AcquisitionData
from cil.utilities.display import show_geometry
from cil.utilities.display import show2D
from zipfile import ZipFile
import numpy as np

Download and extract the file `SparseBeads_B12_L1` from Zenodo [here](https://zenodo.org/records/290117/files/SparseBeads_B12_L1.zip).

In [ ]:
!wget -P ../data https://zenodo.org/records/290117/files/SparseBeads_B12_L1.zip

z = ZipFile(os.path.expanduser("../data/SparseBeads_B12_L1.zip"))
z.extractall(os.path.expanduser("../data/"))
!rm -rf ../data/SparseBeads_B12_L1.zip

First let's load the raw data file and look at its properties:

In [ ]:
file_name = '../data/SparseBeads_B12_L1/CentreSlice/Sinograms/SparseBeads_B12_L1_0001.tif'
data_reader = TIFFStackReader(file_name)
data = data_reader.read()

print(data, "\n", data.shape)

We can see the data is an array of pixel intensities of 2520 projections, containing 2000 horizontal pixels and a single vertical slice. 

We know this dataset has fan-beam geometry, so we create a Cone2D (a.k.a. fan-beam) AcquisitionGeometry object.

We need to specify the `source_position`, `detector_position`, and `detector_direction` used in the experiment.  
For this dataset, this metadata is stored in the file `SparseBeads_B12_L1.xtek2dct`. The location of the metadata may vary across systems and data-saving methods.

The following coordinates describe the source and detector positions:
* `SrcToObject=121.932688713074` 
* `SrcToDetector=1400.207` 

In CIL, the object (sample)'s coordinate is treated as the **0 position**.  
 The source position is therefore 0 - `SrcToObject`.  
 The detector position is `SrcToDetector` - `SrcToObject`.

Finally, we set the `num_pixels` to the number of horizontal pixels, and the `pixel_size`:

In [ ]:
SrcToObject=121.932688713074
SrcToDetector=1400.207

src_coord = 0 - SrcToObject
detec_coord = SrcToDetector - SrcToObject

cone_geom = AcquisitionGeometry.create_Cone2D(source_position= [0, src_coord], 
                                              detector_position= [0, detec_coord], 
                                              detector_direction_x= [1, 0]) \
                                .set_panel(num_pixels=data.shape[1], pixel_size=[0.2, 0.2])

To complete the geometry information, we generate a list of angles based on the number of projections and the `AngularStep`:

In [ ]:
Projections=2520
AngularStep=0.142857142857143

angles = np.linspace(0, Projections*AngularStep, Projections, endpoint=False)

cone_geom.set_angles(angles=angles)

Now we have created our geometry. We can visualise it to check that it looks accurate, with the correct shape and source/detector positions:

In [ ]:
show_geometry(cone_geom)

In CIL, we store the data and the `AcquisitionGeometry` in an `AcquisitionData` object, which is needed to use many of CIL's reconstruction and visualisation tools:

In [ ]:
sparse_beads = AcquisitionData(array=data, geometry=cone_geom)
print(sparse_beads)

Using `show2D()`, we can view a central projection of the data:

In [ ]:
show2D(sparse_beads)

### Checking The Reconstruction
##### Below we use the FDK algorithm to reconstruct this dataset, and check that the geometry is correct:

In [ ]:
from cil.recon import FDK
from cil.processors import TransmissionAbsorptionConverter

# Convert data to absorption data
sparse_beads = TransmissionAbsorptionConverter()(sparse_beads)

# Perform reconstruction
recon = FDK(sparse_beads).run()

# Apply a mask to show the beads only
recon.apply_circular_mask(0.9)


In [ ]:
show2D(recon)

Above we can see that there are double edges over each bead, which indicates that the centre of rotation is slightly off.

We can use the `CentreOfRotationCorrector` processor to correct the centre of rotation offset, and perform the reconstruction again:

In [ ]:
from cil.processors import CentreOfRotationCorrector

processor = CentreOfRotationCorrector.image_sharpness()
processor.set_input(sparse_beads)
centred_data = processor.get_output()

In [ ]:
# Perform reconstruction
recon = FDK(centred_data).run()

# Apply a mask to show the beads only
recon.apply_circular_mask(0.9)

Now the geometry is more accurate, and results in a more reasonable reconstruction without the double edges:

In [ ]:
show2D(recon)